# Note
- This jupyter notebook is designed for named entity recognition. Although some codes were omitted for server security,  all necessary codes are included.
- Point of contact: Jihun Jeung, jihun@gm.gist.ac.kr

# install libarary

In [1]:
import pandas as pd

In [37]:
# !/usr/bin/python3 -m pip install --upgrade pip

In [38]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [39]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-wc2f0lq0/kobert-tokenizer_c8334ac4ce3741518a4c6a241b93439d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-wc2f0lq0/kobert-tokenizer_c8334ac4ce3741518a4c6a241b93439d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit e1f2f37055e7460d8427f6912579c0162cb69831
  Preparing metadata (setup.py) ... done


# data

source : https://github.com/naver/nlp-challenge/blob/master/missions/ner/data/train/train_data

In [41]:
# data = pd.read_csv('data/train_data.txt', sep='\t',  header=None)
# data.columns = ['index' , 'content', 'tag']
# data['word_id'] = [k for k in range(len(data['content']))]

In [3]:
data = pd.read_csv('data/indexingFile.csv', sep=',',  header=0)
data.columns = ['number', 'index' , 'content', 'tag']
data = data.drop(['number'], axis=1)
data['word_id'] = [k for k in range(len(data['content']))]

In [4]:
data.head(20)

,index,content,tag,word_id
0,1,강사,1,0
1,2,활동,0,1
2,3,하면,0,2
3,4,잘할,0,3
4,5,것,0,4
5,6,같은,0,5
6,7,스타는?.,0,6
7,1,커피,1,7
8,2,갤러,0,8
9,3,리,0,9


To split by a sentence

In [5]:
content_split = []
tag_split = []
word_id_split = []

content_spliter = []
tag_spliter = []
word_spliter = []

for i in range(len(data)):
    content_spliter.append(data['content'].iloc[i])
    tag_spliter.append(data['tag'].iloc[i])
    word_spliter.append(data['word_id'].iloc[i])
    if (i == len(data)-1) : # the last word in a data
        content_split.append(content_spliter)
        content_spliter = []
        tag_split.append(tag_spliter)
        tag_spliter = []
        word_id_split.append(word_spliter)
        word_spliter = []
    elif (data['index'].iloc[i+1] == 1) : # the last word in a sentence
        content_split.append(content_spliter)
        content_spliter = []
        tag_split.append(tag_spliter)
        tag_spliter = []
        word_id_split.append(word_spliter)
        word_spliter = []
        
# sanity check
len(content_split) == len(tag_split) == len(word_id_split)

True

In [6]:
len(content_split)

31249

In [7]:
len(tag_split)

31249

In [8]:
len(word_id_split)

31249

## tag integer encodding

In [9]:
tag_list = data['tag'].unique()
num_tags = len(tag_list)
tag_list

array([1, 0])

In [16]:
idex2tag = {}
idex2tag[0]='[UNK]'
idex2tag[1]='[PAD]'
idex2tag[2]='[CLS]'
idex2tag[3]='[SEP]'
idex2tag[4]='[MASK]'

In [17]:
tag2idex = {'[UNK]':0,'[PAD]':1,'[CLS]':2,'[SEP]':3,'[MASK]':4, 0:5, 1:6}

In [19]:
# seq_to_index function
def tag_to_index(sequence, embedding_index):
    '''
    INPUT
        sequence : string list
    OUTPUT
        integer list
    '''
    converted_index = [embedding_index[word] if word in embedding_index.keys() else embedding_index['[UNK]'] for word in sequence]
    return converted_index

# index2gene
idex2tag = {v:k for k,v in tag2idex.items()}
idex2tag[0]='[UNK]'
idex2tag[1]='[PAD]'
idex2tag[2]='[CLS]'
idex2tag[3]='[SEP]'
idex2tag[4]='[MASK]'
#print(list(gene2idex.keys())[list(gene2idex.values()).index(16)]) 
print(f'{idex2tag[1]}')

print(tag_to_index(['5', '6'], tag2idex)) # example

[PAD]
[0, 0]


## tokenizer

- [PAD] : 1
- [CLS] : 2
- [SEP] : 3
- [MASK] : 4

In [21]:
from kobert_tokenizer import KoBERTTokenizer
import gluonnlp as nlp
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [22]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [23]:
tokenizer.encode("[UNK]")

[2, 0, 3]

In [24]:
tokenizer.encode("[PAD]")

[2, 1, 3]

In [25]:
tokenizer.encode("[CLS]")

[2, 2, 3]

In [26]:
tokenizer.encode("[SEP]")

[2, 3, 3]

In [27]:
tokenizer.encode("[MASK]")

[2, 4, 3]

example

In [ ]:
data['content'][0]

'강사'

In [ ]:
type(data['content'][0:5])

pandas.core.series.Series

In [ ]:
list(data['content'][0:5])

['강사', '활동', '하면', '잘할', '것']

In [ ]:
text = ' '.join(list(data['content'][0:20]))

In [ ]:
tokenizer.encode(text)

[2,
 807,
 6493,
 5141,
 4939,
 3942,
 7836,
 905,
 833,
 2938,
 5760,
 258,
 54,
 4656,
 517,
 5371,
 6037,
 1900,
 4103,
 2044,
 7089,
 54,
 4656,
 517,
 5371,
 3831,
 1182,
 522,
 4213,
 5960,
 1115,
 7119,
 4257,
 40,
 54,
 3]

In [ ]:
[tokenizer.decode(token) for token in tokenizer.encode(text)]

['[CLS]',
 '강',
 '사',
 '활동',
 '하면',
 '잘',
 '할',
 '것',
 '같은',
 '스타',
 '는',
 '?',
 '.',
 '커피',
 '',
 '갤',
 '러',
 '리',
 '정보',
 '모',
 '음',
 '.',
 '커피',
 '',
 '갤',
 '임시',
 '규정',
 '(',
 '주',
 '딱',
 '구',
 '인',
 '중',
 ')',
 '.',
 '[SEP]']

In [ ]:
data

,index,content,tag,word_id
0,1,강사,1,0
1,2,활동,0,1
2,3,하면,0,2
3,4,잘할,0,3
4,5,것,0,4
...,...,...,...,...
177392,3,게샤,1,177392
177393,4,6,1,177393
177394,5,-GW,0,177394
177395,6,워시드,1,177395


In [ ]:
len(data)

177397

In [ ]:
data['content'].iloc[0]

'강사'

In [ ]:
tokenizer.encode(data['content'].iloc[0])

[2, 807, 6493, 3]

In [ ]:
data['word_id']

0              0
1              1
2              2
3              3
4              4
           ...  
177392    177392
177393    177393
177394    177394
177395    177395
177396    177396
Name: word_id, Length: 177397, dtype: int64

In [ ]:
data['word_id'][0:10]

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
Name: word_id, dtype: int64

In [ ]:
data['tag'].iloc[0]

1

In [ ]:
tag2idex[data['tag'].iloc[0]]

6

In [43]:
def tokenizer_per_sentece(content_list, tag_list, word_id_list):
    """
    INPUT:
        content_split_list : a list that represents a sentence 
            e.g. ['비토리오', '양일', '만에', '영사관', '감호', '용퇴,', '항룡', '압력설', '의심만', '가율']
        tag_split_list
            e.g. ['PER_B', 'DAT_B', '-', 'ORG_B', 'CVL_B', '-', '-', '-', '-', '-']
        word_id_split_list
            e.g. [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    OUTPUT: tokenzier results
        content_tokenizer : a tokenized content list without special tokens
        word_id_tokenizer : a word id list
        tag_id_tokenizer : a tag id list that the only first content has a tag.
    """
    # sanity check
    if not (len(content_list)==len(tag_list)==len(word_id_list)):
        print("check the input data")
    
    # tokenizer
    content_tokenizer = []
    word_id_tokenizer = []
    tag_id_tokenizer = []
    for i in range(len(word_id_list)):
        # tokenizer
        content_tokenizer += tokenizer.encode(content_list[i])[1:-1] # to remove special tokens [CLS] and [SEP]
        word_id_tokenizer += [word_id_list[i]] * (len(tokenizer.encode(content_list[i]))-2)
        tag_id_tokenizer += [tag2idex[tag_list[i]]] + [-100] * (len(tokenizer.encode(content_list[i]))-3)
    
    if not (len(content_tokenizer) == len(word_id_tokenizer) == len(tag_id_tokenizer)):
        print(f'sanity check : {len(content_tokenizer) == len(word_id_tokenizer) == len(tag_id_tokenizer)}')
    return content_tokenizer, tag_id_tokenizer, word_id_tokenizer

In [44]:
a, b, c = tokenizer_per_sentece(content_split[300], tag_split[300], word_id_split[300])

In [45]:
data_tokenizer = pd.DataFrame({'token': a, 'tag_id': b, 'word_ids': c})
data_tokenizer

,token,tag_id,word_ids
0,517,6,1692
1,7490,-100,1692
2,7491,-100,1692
3,6392,-100,1692
4,3084,5,1693
5,6198,-100,1693
6,3537,5,1694
7,517,5,1695
8,54,-100,1695


In [46]:
from tqdm import tqdm 
def tokenizer_all_data(content_split, tag_split, word_id_split):
    # sanity check
    if not (len(content_split)==len(tag_split)==len(word_id_split)):
        print("check the input data")
        
    content_split_token, tag_split_token, word_id_split_token = [], [], []
    for i in tqdm(range(len((content_split)))):
        content, tag, wordID = tokenizer_per_sentece(content_split[i], tag_split[i], word_id_split[i])
        content_split_token.append(content)
        tag_split_token.append(tag)
        word_id_split_token.append(wordID)
    return content_split_token, tag_split_token, word_id_split_token

In [47]:
content_split_tok, tag_split_tok, word_id_split_tok = tokenizer_all_data(content_split, tag_split, word_id_split)

100%|████████████████████████████████████| 31249/31249 [00:32<00:00, 967.02it/s]


In [49]:
#example
[tokenizer.decode(token) for token in content_split_tok[0]]

['강', '사', '활동', '하면', '잘', '할', '것', '같은', '스타', '는', '?', '.']

Padding

In [ ]:
print(max([len(x) for x in content_split_tok]))
print(sum([len(x) for x in content_split_tok])/len(content_split_tok))

40
12.790169285417134


In [51]:
def add_padding(token_list, max_len=512):
    '''
    INPUT
        token_list : list
            e.g. [[1,2,3,4], [5,6,7,8]]
        max_len : int
    OUTPUT
        token_ls : list
    '''
    import copy
    token_ls = copy.deepcopy(token_list)
    max_len = max_len-2 # to consider the special tokens
    for index, tokens in enumerate(token_ls):
        n_token = len(tokens)
        pad = 1 # an index of special token
        bos = 2 # an index of special token
        eos = 3 # an index of special token
        # add padding if the length is shorter than max_len
        if n_token < max_len:
            token_ls[index] += [pad] * (max_len - n_token) # 부족한 만큼 padding을 추가함
        
        # if a sentence is longer than max_len, cut a sentence.
        elif n_token > max_len:
            token_ls[index] = tokens[:max_len]
        
        # to add begining token and end token
        token_ls[index] = [bos] + token_ls[index] + [eos]
    return token_ls

In [52]:
content_input = add_padding(content_split_tok, 512)
tag_input = add_padding(tag_split_tok, 512)
word_id_input = add_padding(word_id_split_tok, 512)

In [ ]:
# tag_input[100]

# To load data

# data loader

In [16]:
input_x = torch.tensor(content_input)
input_y = torch.tensor(tag_input)

In [19]:
from torch.utils.data import TensorDataset 
from torch.utils.data import DataLoader
from torch.utils.data import random_split

batch_size=8
total_dataset = TensorDataset(input_x, input_y)

num_train = int(len(total_dataset)*0.8)
num_validation = int(len(total_dataset)*0.1)
num_test = len(total_dataset)-num_train-num_validation

train_dataset, validation_dataset, test_dataset = random_split(total_dataset, [num_train, num_validation, num_test])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


print(len(total_dataset))
print(len(train_dataset))
print(len(validation_dataset))
print(len(test_dataset))
print('sanity check: ', len(total_dataset)==len(train_dataset)+len(validation_dataset)+len(test_dataset))


len(train_loader)
len(validation_loader)
len(test_loader)

del total_dataset, num_train, num_validation, num_test, train_dataset, validation_dataset, test_dataset

31249
24999
3124
3126
sanity check:  True


# NER model

In [8]:
max_length = seq_length = 512
num_tag = 7

In [124]:
import torch
from transformers import BertModel
pretrained_model = BertModel.from_pretrained('skt/kobert-base-v1')

In [125]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

class KoBERTNER(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, #768
                 num_labels=7,
                 max_length=512, 
                 dr_rate=None,
                 params=None):
        super(KoBERTNER, self).__init__()
        self.bert = bert # Load model body
        self.dr_rate = dr_rate
        self.max_length = max_length
        # Set up token classification head
        self.num_labels = num_labels # num_tags
        self.classifier = nn.Linear(hidden_size, self.max_length*self.num_labels)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
            
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask) #[batch_size, hidden_dim]
            # ouitput[0] : the attention weights
            # output[1] : the output from the model
        # Apply classifier to encoder representation
        output = self.dropout(output[1])
        output = self.classifier(output) #[batch_size, max_length*num_label]
        output = output.view([-1, self.num_labels, self.max_length]) # [batch_size, num_label, max_length]
        return output

In [126]:
DEVICE = "cuda"
model = KoBERTNER(pretrained_model, dr_rate=0.3).to(DEVICE)

In [12]:
def train_epoch(model, optimizer, data_loader, max_length=max_length, DEVICE='cuda'):
    import gc
    from tqdm import tqdm
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    losses = 0
    num_correct = []
    num_test = 0
    torch.autograd.set_detect_anomaly(True)
    for src, tgt in data_loader:
        src = src.type(torch.LongTensor).to(DEVICE)
        tgt = tgt.type(torch.LongTensor).to(DEVICE)
        logits = model(input_ids= src, attention_mask= torch.tensor([[1]*max_length]*batch_size).to(DEVICE))
            #torch.Size([batch_size, , num_tag, src_seq_length])
        optimizer.zero_grad()
        loss = loss_fn(logits, tgt)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        losses += loss.item()
        
        # accuracy
        _, pred_word = torch.max(logits, dim=1)
        num_correct.append(int((torch.flatten(tgt)==torch.flatten(pred_word)).sum()))
        num_test += int(torch.flatten(pred_word).size(0))
        
    accuracy = sum(num_correct) / num_test
    
    # garbage collector
    gc.collect()
    torch.cuda.empty_cache()
 
    return losses / len(data_loader), accuracy


def test_epoch(model, optimizer, data_loader, max_length=max_length, DEVICE='cuda'):
    import gc
    import numpy as np
    with torch.no_grad():
        model.eval()
        loss_fn = torch.nn.CrossEntropyLoss()
        losses = 0
        num_correct = []
        num_test = 0
        y_probability = np.array([])
        y_pred = np.array([])
        y_true = np.array([])
        for src, tgt in data_loader:
            src = src.type(torch.LongTensor).to(DEVICE)
            tgt = tgt.type(torch.LongTensor).to(DEVICE)
            logits = model(input_ids= src, attention_mask= torch.tensor([[1]*max_length]*batch_size).to(DEVICE))
                #torch.Size([batch_size, , num_tag, src_seq_length])
            
            loss = loss_fn(logits, tgt)
            losses += loss.item()
            
            # accuracy
            _, pred_word = torch.max(logits, dim=1)
            num_correct.append(int((torch.flatten(tgt)==torch.flatten(pred_word)).sum()))
            num_test += int(torch.flatten(pred_word).size(0))
            
            y_probability = np.append(y_probability, logits.cpu().detach().numpy())
            y_pred = np.append(y_pred, pred_word.cpu().detach().numpy())
            y_true = np.append(y_true, tgt.cpu().detach().numpy())
            
    accuracy = sum(num_correct) / num_test
    # garbage collector
    gc.collect()
    torch.cuda.empty_cache()
    
    return losses / len(data_loader), accuracy,  y_pred, y_true

In [ ]:
def train_all_sample(model, optimizer, max_length, train_loader, validation_loader, test_loader, device, total_num_epoch, running_num_epoch, tf_board_directory, model_save_directory, best_model_save_directory=None, slack_message=False):
    '''
    Parameters:
        train_loader
        validation_loader
        test_loader
        total_num_epoch : the total number of epoch that have run
        running_num_epoch : the number of epoch that run in this time
        model_save_directory : save directory to save the final model
        best_model_save_directory : [optional] save directory to save the best model (best validation accuracy)
    '''
  

    if (total_num_epoch < 0 or running_num_epoch <= 0):
        import sys
        sys.exit("Check the number of epoch. It is incorrect")

    import time
    from tqdm import tqdm
    import numpy as np
    total_time_start = time.time() # to measure time
    best_validation_accuracy = None
    for epoch in tqdm(range(1, running_num_epoch+1)):
        epoch_time_start = time.time() # to measure time
        lr = scheduler.optimizer.param_groups[0]['lr']
        loss, train_accuracy = train_epoch(model, optimizer, train_loader, max_length=max_length)
        _, validation_accuracy, _, _ = test_epoch(model, optimizer, validation_loader, max_length=max_length)

        # to save the metrics and model
        if best_validation_accuracy is None or validation_accuracy >= best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            if not(best_model_save_directory is None):            
                torch.save({
                'epoch': total_num_epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, best_model_save_directory)
        _, test_accuracy, _, _ = test_epoch(model, optimizer, test_loader, max_length=max_length)
        total_num_epoch = total_num_epoch + 1
        epoch_time_end = time.time() # to measure time    
   
    total_time_finish = time.time() # to measure time
    print(f'Done. Total running Time: {total_time_finish - total_time_start}')
    writer.close() #tensorboard : if close() is not declared, the writer does not save any valeus.

    # model save
    torch.save({
            'epoch': total_num_epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, model_save_directory)
    print('total number of epoches : ', total_num_epoch)
    print("-------------------------done------------------------------")

In [ ]:
save_directory = 'run_00'
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.8, patience=10, min_lr=1e-4)

train_all_sample(model=model,
                optimizer=optimizer,
                max_length=max_length,
                train_loader=train_loader, 
                validation_loader=validation_loader, 
                test_loader=test_loader, 
                device='cuda', 
                total_num_epoch=0, 
                running_num_epoch=300, 
                tf_board_directory = 'tfboard/'+save_directory, 
                model_save_directory='model/'+save_directory, 
                best_model_save_directory='model/'+save_directory+'_best',
                slack_message = True)
del save_directory
slack_alarm('The program execution on JupyterNotebook is done')

# To load model

In [5]:
import torch
from transformers import BertModel
pretrained_model = BertModel.from_pretrained('skt/kobert-base-v1')

In [10]:
DEVICE = "cuda"
model = KoBERTNER(pretrained_model, dr_rate=0.3).to(DEVICE)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0015334791245047435)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.8, patience=5, min_lr=0.00001)
#optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
PATH = 'model/run_00_best'

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [21]:
loss, accuracy, pred, true = test_epoch(model=model, optimizer=optimizer, data_loader= test_loader, max_length=max_length, DEVICE='cuda')

In [32]:
loss

0.019896473274685633

In [33]:
pred[2000:3000]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 2., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [31]:
true[1000:2000]

array([   1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    3.,    2.,    6., -100.,
       -100., -100.,    6., -100., -100.,    6.,    6.,    5., -100.,
       -100., -100., -100., -100.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          1.,    1.,